# Data Cleaning and Setup

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn
seaborn.set(style='ticks')
import warnings
warnings.filterwarnings('ignore')
 
# Definfe the stock and ETFs universe
ETFs_universe = ['MGC', 'MGK', 'MGV', 'MTUM', 'QQQ', 'SDY', 'SPLV', 'VB', 
                 'VBK', 'VBR', 'VCR', 'VDC', 'VDE', 'VFH', 'VGT', 'VHT', 'VIG',
                 'VIS', 'VNQ', 'VO', 'VOE', 'VOT', 'VOX', 'VPU', 'VTV', 'VUG',
                 'VV', 'VYM', 'XBI', 'XLB', 'XLC', 'XLI', 'XLK', 'XLV', 'XME']


# Extract historiacl data from IEX
from iexfinance.stocks import get_historical_data
from iexfinance.stocks import Stock
from datetime import datetime
from datetime import timedelta
import os

# Authenticate the IEX with token
os.environ['IEX_API_VERSION'] = 'iexcloud-sandbox'
os.environ['IEX_TOKEN'] = 'Tpk_xxxxxxxx' # put your token here

# Fetch the data from IEX
start_date = datetime.now() - timedelta(days = 365) # since our rebalance frequency is high, we won't need too much data here.
data = []

df = pd.DataFrame(columns=['symbol', 'fClose', 'changePercent'])
return_table = pd.DataFrame()

# Fetch the data

In [2]:
for s in ETFs_universe:
    df_extract = get_historical_data(s, start_date).loc[:, ['symbol', 'fClose', 'changePercent']]
    return_data = pd.to_numeric(df_extract.changePercent)
    return_table[s] = return_data
    df = df.append(df_extract)
    
# Change datatype into numeric
df.changePercent = pd.to_numeric(df.changePercent)
df.fClose = pd.to_numeric(df.fClose)
df.head()

,symbol,fClose,changePercent
2020-05-21,MGC,104.8728,-0.0085
2020-05-22,MGC,106.6988,0.0024
2020-05-26,MGC,104.9400,0.0113
2020-05-27,MGC,105.5799,0.0143
2020-05-28,MGC,109.7830,-0.0022


# Momentum Filter

In [3]:
return_table.head()

,MGC,MGK,MGV,MTUM,QQQ,SDY,SPLV,VB,VBK,VBR,...,VUG,VV,VYM,XBI,XLB,XLC,XLI,XLK,XLV,XME
2020-05-21,-0.0085,-0.0085,-0.0072,-0.0096,-0.0112,-0.0001,-0.0094,0.0020,-0.0007,0.0047,...,-0.0080,-0.0068,-0.0070,-0.0081,-0.0098,-0.0074,0.0024,-0.0139,-0.0077,-0.0152
2020-05-22,0.0024,0.0031,0.0006,0.0048,0.0036,-0.0019,0.0036,0.0046,0.0075,0.0025,...,0.0044,0.0024,-0.0009,0.0120,-0.0016,0.0060,-0.0008,0.0037,0.0022,-0.0097
2020-05-26,0.0113,0.0012,0.0237,-0.0079,-0.0028,0.0369,0.0082,0.0280,0.0132,0.0429,...,0.0024,0.0118,0.0230,-0.0240,0.0286,0.0029,0.0430,-0.0010,-0.0018,0.0183
2020-05-27,0.0143,0.0062,0.0236,0.0074,0.0057,0.0267,0.0202,0.0249,0.0134,0.0346,...,0.0068,0.0146,0.0213,0.0003,0.0102,0.0072,0.0350,0.0059,0.0109,0.0385
2020-05-28,-0.0022,0.0017,-0.0040,0.0024,-0.0013,-0.0105,0.0100,-0.0150,-0.0064,-0.0220,...,0.0020,-0.0009,-0.0031,-0.0100,0.0096,-0.0095,-0.0098,-0.0020,0.0140,-0.0170


In [4]:
def momemtum_ranking_table(df, t, n):
    # lookback period = t weeks
    # select top n stock
    
    # create a ranking table (smaller the ranking, higher the return)
    ranking = resample_risk_adjusted_retrun_table(df, t).shift(1).rank(axis=1, na_option='keep', ascending = False)

    original_rank = list(range (1, len(ETFs_universe)+1))
    replace_rank = [1]*n + [0]*(len(ETFs_universe)-n)

    ranking_matrix = ranking.replace(original_rank, replace_rank)# replace_rank, make the value of top n as 1, all others as 0
    
    return ranking_matrix

def cumulative_return(return_vals):
    # convert daily retrun into any cumulative return we want
    
    cumulative_return_list = (np.array(return_vals)+1).cumprod() -1
    
    return cumulative_return_list[-1]

def resample_retrun_table(df, t):
    # for actual backtesting, we need the actual retrun(un-risk-adjusted)
    
    # df is the daily return table (the most recent datetime should at the bottom of the table)
    # t is the number of week we want to resample into
    # W-Fri: W means weekly, while Fri means use Friday to represent the week
    # We rebalance the position on the close price of Friday
    
    df = df.resample("W-Fri").apply(lambda x : cumulative_return(x)) # turn the dataframe into weekly 
    index = df.index # store the weekly datetime index
    df = df.reset_index().drop(columns=['index'])

    rolling_return = (1 + df).rolling(window = t).apply(np.prod, raw = True) - 1
    rolling_return = rolling_return.set_index([index])
    
    return rolling_return.iloc[::-1].iloc[::t, :].iloc[::-1].iloc[1:] # turn the table upside dowm, then select every t-th row, the turn around the table again 

def resample_risk_adjusted_retrun_table(df, t):
    # for ranking propose, we need risk adjusted return
    
    df = df.resample("W-Fri").apply(lambda x : cumulative_return(x))
    risk = df.rolling(window=5*t).std()# turn the dataframe into weekly 
    index = df.index # store the weekly datetime index
    df = df.reset_index().drop(columns=['index'])

    rolling_return = (1 + df).rolling(window = t).apply(np.prod, raw = True) - 1
    rolling_return = rolling_return.set_index([index])
    resample_return_table = rolling_return.iloc[::-1].iloc[::t, :].iloc[::-1].iloc[1:]
    
    return resample_return_table.mul(1/risk).dropna(how='all') # multiply element-wise

def portfolio_retrun_table(df, t, n):
    return momemtum_ranking_table(df, t, n).mul(resample_retrun_table(df, t)).sum(1)*(1/n) # equal weighted here

def plot_maximum_drawdown(df, t, n):
    wealth_index=(1+portfolio_retrun_table(df,t,n)).cumprod()
    plt.plot(wealth_index)
    previous_peaks = pd.Series(wealth_index).cummax()
    previous_peaks.plot()

In [5]:
resample_risk_adjusted_retrun_table(return_table, 3)

,MGC,MGK,MGV,MTUM,QQQ,SDY,SPLV,VB,VBK,VBR,...,VUG,VV,VYM,XBI,XLB,XLC,XLI,XLK,XLV,XME
2020-09-11,-0.719365,-1.012177,-0.148839,-1.565194,-1.289450,-0.257807,-0.785250,-0.830550,-1.408500,-0.324232,...,-1.042545,-0.601570,-0.209491,-1.732537,1.323706,-0.512790,0.335103,-1.398954,-0.766959,-0.200692
2020-10-02,0.218727,0.453855,-0.347549,1.142849,0.470728,-0.439386,0.370057,1.206001,2.343439,0.303366,...,0.559497,0.219749,-0.627225,1.288855,-0.942415,-0.377285,0.017837,0.651374,-0.165326,0.081125
2020-10-23,1.587159,1.273379,1.655701,1.070677,1.411631,2.096973,1.629959,2.448907,2.334433,2.196288,...,1.494813,1.797401,1.744162,1.371365,1.591297,2.333170,1.801386,0.948440,1.761047,1.457876
2020-11-13,1.037433,0.400118,1.771444,0.298977,0.541456,1.657781,0.680067,1.367962,1.053857,1.642405,...,0.444583,1.030740,1.622955,1.292878,1.469210,1.011100,1.571024,0.487925,1.271547,1.624704
2020-12-04,1.114163,0.827423,1.399389,1.045893,1.333326,0.976976,-0.102976,2.153708,1.675436,2.265165,...,1.025811,1.314635,1.645402,1.850388,1.312693,1.407752,1.191973,0.981316,0.110947,3.366534
2020-12-25,0.031774,0.406774,-0.555032,0.959474,0.419237,-0.534869,0.185544,1.245555,2.314731,0.303422,...,0.558117,0.089916,-0.651888,2.242019,-0.081609,-0.409978,-0.398433,0.658622,-0.303507,-0.042450
2021-01-15,0.480880,-0.241807,1.548428,0.508523,0.219460,1.321154,0.327161,1.646121,1.018130,1.980818,...,-0.093732,0.579216,1.821392,0.306046,1.754690,-0.503356,0.293899,-0.372748,1.544979,2.004497
2021-02-05,1.128200,1.765834,-0.012498,1.473906,1.709720,0.029150,0.223161,0.916099,1.111552,0.534249,...,1.593339,1.113417,-0.050334,2.502328,-0.719574,2.573129,0.045031,1.607050,-0.371491,-0.410214
2021-02-26,-1.151580,-1.791315,0.346994,-1.930571,-1.796531,0.722515,-2.467504,-0.189302,-1.159397,1.093251,...,-1.631978,-1.028632,0.330430,-2.569781,0.027766,0.092415,0.924083,-1.240721,-1.361360,0.831263
2021-03-19,1.173158,0.112929,2.638145,-0.748640,-0.118164,2.410985,2.674743,1.069918,-0.433169,2.510227,...,-0.087581,1.099766,2.701202,-0.357245,2.128021,1.988751,2.949156,-0.319619,1.302837,2.389637


In [6]:
# find the winners

current_ranking = momemtum_ranking_table(return_table, 1, 5).iloc[-1]
winners = current_ranking[current_ranking == 1].index.tolist()
winners

['VDC', 'VDE', 'VFH', 'VYM', 'XLB']

# Risk Budgeting

In [7]:
df_return = pd.DataFrame(columns=winners)

for s in winners:
    df_return[s] = df.loc[df.symbol == s].changePercent

V = df_return.cov().values

In [8]:
# set the target risk and solve the weight for each stock
# w is the weight vector
# V is the covariace matrix

from __future__ import division
from matplotlib import pyplot as plt
from numpy.linalg import inv,pinv
from scipy.optimize import minimize
from scipy.optimize import basinhopping

# risk budgeting optimization
def calculate_portfolio_var(w,V):
    # function that calculates portfolio risk
    w = np.matrix(w)
    return (w*V*w.T)[0,0]

def calculate_risk_contribution(w,V):
    # function that calculates asset contribution to total risk
    w = np.matrix(w)
    sigma = np.sqrt(calculate_portfolio_var(w,V))
    # Marginal Risk Contribution
    MRC = V*w.T
    # Risk Contribution
    RC = np.multiply(MRC,w.T)/sigma
    return RC

def risk_budget_objective(x,pars):
    # calculate portfolio risk
    V = pars[0] # covariance table
    x_t = pars[1] # risk target in percent of portfolio risk
    sig_p =  np.sqrt(calculate_portfolio_var(x,V)) # portfolio sigma
    risk_target = np.asmatrix(np.multiply(sig_p,x_t))
    asset_RC = calculate_risk_contribution(x,V)
    J = sum(np.square(asset_RC - risk_target.T))[0,0] # sum of squared error
    return J*100

def total_weight_constraint(x):
    return np.sum(x)-1.0

def long_only_constraint(x):
    return x

In [9]:
# setup the target risk contribution and constriants
number_of_sercurities = len(winners)
x = number_of_sercurities

w0 = [1/x]*x # initial guess
x_t = [1/x]*x # your risk budget percent of total portfolio risk (equal weight here)
cons = ({'type': 'eq', 'fun': total_weight_constraint},{'type': 'ineq', 'fun': long_only_constraint})
res= basinhopping(risk_budget_objective, w0, minimizer_kwargs={"method":"SLSQP","args":[V,x_t],"constraints":cons}, 
                  niter = 1000, stepsize = 0.001) # basinhopping model
w_rb = np.asmatrix(res.x)
w_rb # target position weight

matrix([[0.34333481, 0.10679011, 0.15392988, 0.21084427, 0.18510093]])

In [10]:
# Set the notional capital and target yearly volatility here
notional_capital = 100000
target_annualized_volatility = 0.2

target_daily_volatility = target_annualized_volatility/math.sqrt(252)
target_daily_money_risk = notional_capital * target_daily_volatility

real_portfolio_SD = math.sqrt((w_rb*V*w_rb.T)[0,0])
real_portfolio_value = target_daily_money_risk / real_portfolio_SD

# Output the target portfolio
from datetime import date
today = date.today() - timedelta(days = 1)
today = today.strftime('%Y-%m-%d')

df_today = df.loc[today]
price_list = []

target_portfolio = pd.DataFrame()  
target_portfolio['Ticker'] = winners

for ticker in winners:
    price = list(df_today.loc[df_today['symbol'] == ticker].fClose)[0]
    price_list.append(price)
  
target_portfolio['Price'] = price_list
target_portfolio['Weight'] = np.array(list(np.array(w_rb[0])[0]))
target_portfolio['Value'] = target_portfolio.Weight * real_portfolio_value
target_portfolio['Quantity'] = target_portfolio['Value'] / target_portfolio['Price']
target_portfolio.round(2).reset_index(drop=True)

,Ticker,Price,Weight,Value,Quantity
0,VDC,187.74,0.34,36517.92,194.51
1,VDE,73.67,0.11,11358.45,154.18
2,VFH,91.94,0.15,16372.35,178.08
3,VYM,110.30,0.21,22425.90,203.32
4,XLB,89.82,0.19,19687.78,219.19


In [11]:
# real portfolio value
target_portfolio.Value.sum().round()

106362.0